In [ ]:
!pip install -q snscrape==0.3.4
#!pip install lmfit

In [ ]:
import os
import math
import pandas as pd
import numpy as np
from numpy import cumsum
from sklearn import metrics 
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
import networkx as nx
from scipy import integrate, optimize
from numpy import linalg as la
from datetime import date, datetime, timedelta
from IPython import display
from google.colab import drive
drive.mount('/drive')

# Data Collection

In [ ]:
def get_total_tweets(search_term, start_date, end_date):
  os.system(f"snscrape --since {start_date} twitter-search '{search_term} until:{end_date}' > result-tweets-{start_date}-{end_date}.txt")
  if os.stat(f"result-tweets-{start_date}-{end_date}.txt").st_size == 0:
    counter = 0
  else:
    df = pd.read_csv(f"result-tweets-{start_date}-{end_date}.txt", names=['link'])
    total_tweets = df.size

  return total_tweets, df

def get_cumulative_tweets(arr):
  cumulative_tweets = cumsum(arr)
  return cumulative_tweets

def generate_dataset(search_term, total_days):

  # # # # # # # # 
  # Generic code to specify an arbitary time window in (today - total_days, today)
  # # # # # # # # 
  #
  # dates = []
  # today_date = date.today()
  # start_date = today_date - timedelta(days = total_days) 

  # for i in range(total_days):
  #   curr_date = today_date - timedelta(days = i)
  #   dates.append(curr_date)

  # dates.append(start_date)

  # dates.reverse()
  # for i in range(len(dates)):
  #   dates[i] = dates[i].strftime("%Y-%m-%d")
  #
  # # # # # # # # 

  # To maintain consistent results reproduction by TAs,
  # we are fixing the data collection window to return
  # consistent data used for modeling. We use a start
  # date of 2022-2-21 and collect tweets till 2022-04-28

  dates = []
  total_days = 66
  start_date = date(2022, 4, 28)

  for i in range(total_days):
    curr_date = start_date - timedelta(days = i)
    dates.append(curr_date)

  dates.append(start_date-timedelta(days=total_days))

  dates.reverse()
  for i in range(len(dates)):
    dates[i] = dates[i].strftime("%Y-%m-%d")


  dfs = []
  total_tweets = []
  for i in range(1,len(dates)):
    tweets, df = get_total_tweets(search_term, dates[i-1], dates[i])
    total_tweets.append(tweets)
    dfs.append(df)
    print("Tweet scraping from day",i,"done")
  
  return total_tweets, dfs

In [ ]:
total_days = 66 
search_term = '#IStandWithPutin'
total_tweets, dfs = generate_dataset(search_term, total_days)

In [ ]:
total_tweets

In [ ]:
cumulative_tweets = get_cumulative_tweets(total_tweets)
cumulative_tweets

# Helpers

In [ ]:
def seiz(y, t, beta, b, p, l, rho, eps):
  """
  dS - Rate of change of susceptible users with time
  dE - Rate of change of exposed users with time
  dI - Rate of change of infected users with time
  dZ - Rate of change of skeptics users with time
  """
  dS = -beta*y[0]*(y[2]/N) - b*y[0]*(y[3]/N)
  dE = (1-p)*beta*y[0]*(y[2]/N) + (1-l)*b*y[0]*(y[3]/N) - rho*y[1]*(y[2]/N) - eps*y[1]
  dI = p*beta*y[0]*(y[2]/N) + rho*y[1]*(y[2]/N) + eps*y[1]
  dZ = l*b*y[0]*(y[3]/N)

  return dS, dE, dI, dZ

def fit_seiz(t, beta, b, p, l, rho, eps):
  return integrate.odeint(seiz, (ICs), t, args=(beta, b, p, l, rho, eps))[:,1]

def get_params(f, xdata, ydata,bounds=True):
  if bounds == True:                                                
    return optimize.curve_fit(fit_seiz, xdata, ydata, 
        bounds=([1,1,0,0,10e-05,0], [np.inf, np.inf, 1, 1, 10e-02, 1]))
  else:
    return optimize.curve_fit(fit_seiz, xdata, ydata)

def plot_lsq(ICs, xdata, ydata):
  popt, pcov = get_params(fit_seiz, xdata, ydata)
  fitted = fit_seiz(xdata, *popt)
  fig = plt.figure(figsize=(4,3))
  plt.plot(xdata, ydata, 'o')
  plt.plot(xdata, fitted)
  plt.legend(['Truth', 'Fitted'], loc = 'lower right')
  plt.xlabel("Days", fontsize=15)
  plt.ylabel("Cumulative Tweets", fontsize=15)
  r2_sc = metrics.r2_score(ydata, fitted)
  print("R2_score=", r2_sc)
  return plt

def model(x, t):

  """
  SEIZ ODE model
  """

  beta = popt[0]
  b = popt[1]
  p = popt[2]
  l = popt[3]
  rho = popt[4]
  eps = popt[5]

  S = x[0]
  E = x[1]
  I = x[2]
  Z = x[3]

  dSdt = -beta*S*(I/N) - b*S*(Z/N)
  dEdt = (1-p)*beta*S*(I/N) + (1-l)*b*S*(Z/N) - rho*E*(I/N) - eps*E
  dIdt = p*beta*S*(I/N) + rho*E*(I/N) + eps*E
  dZdt = l*b*S*(Z/N)

  return [dSdt, dEdt, dIdt, dZdt]

def plot_seiz(ICs, xdata):
  """
  ICs: initial conditions for states
  xdata: time data
  """
  sol = integrate.odeint(model,ICs,xdata)
  fig = plt.figure(figsize=(4,3))
  plt.plot(xdata,sol)
  plt.legend(['S','E','I','Z'])
  plt.xlabel("Days", fontsize=15)
  plt.ylabel("Cumulative Tweets", fontsize=15)
  return plt

def plot_animation(sol):

  """
  sol: ODE solution obtained from integrate.odeint or simulation
  """

  fig = plt.figure(figsize=(8,6))
  #ax = plt.axes(xlim=(0, 50), ylim=(0,sol.max()+(0.1*sol.max())))
  ax = plt.axes(xlim=(0, sol.shape[0]), ylim=(0,sol.max()+(0.1*sol.max())))
  plt.xlabel("Days", fontsize=15)
  plt.ylabel("Cumulative Tweets", fontsize=15)
  lines = [plt.plot([], [])[0] for _ in range(4)] 
  patches = lines  

  def init():
      for line in lines:
          line.set_data([], [])

      return patches 

  def animate(i):
      for j,line in enumerate(lines):
          line.set_data(xdata[:i], sol[:i,j])
      plt.legend(['S','E','I','Z'])
      return patches

  anim = animation.FuncAnimation(fig, animate, init_func=init,
                                frames=100, interval=50, blit=True)
  video = anim.to_html5_video()
  html = display.HTML(video)
  return display.display(html)

# Params Fit

In [ ]:
###### Cached values for easy simulation to avoid spending time in scraping. Uncomment values and run the code below for results
# total_days = 65
# cumulative_tweets = np.array([ 29,    35,   182,   401,   569,   765,  1119,  1466, 14788,
#        30574, 33275, 34920, 36236, 37187, 37866, 38367, 38816, 39181,
#        39485, 39758, 40024, 40317, 40646, 40951, 41211, 41450, 41679,
#        41868, 42077, 42234, 42416, 42550, 42742, 42884, 43002, 43105,
#        43241, 43407, 43540, 43657, 43798, 43907, 44031, 44109, 44200,
#        44273, 44368, 44458, 44535, 44621, 44697, 44751, 44817, 44875,
#        44946, 45052, 45129, 45190, 45265, 45322, 45402, 45485, 45551,
#        45607, 45661])

In [ ]:
xdata = np.array([x for x in range(total_days)])
ydata = cumulative_tweets  # first 5 days tweets are low

ydata = np.array(ydata, dtype=float)
xdata = np.array(xdata, dtype=float)

# initial conditions
N = 100000
I0 = ydata[0]
E0 = 100
Z0 = 100
S0 = N - I0 - E0 - Z0
R0 = 0
ICs = [S0, I0, E0, Z0]

popt, pcov = get_params(fit_seiz, xdata, ydata, bounds = False)
popt, pcov = get_params(fit_seiz, xdata, ydata)

In [ ]:
plot_lsq(ICs, xdata, ydata)

In [ ]:
plot_seiz(ICs,xdata)

# Solution Animation

In [ ]:
sol = integrate.odeint(model,ICs,xdata)
plot_animation(sol)